## Pre-processing using spaCy and regex, splitting data into train/test data and external validation data

Connect to drive, set directory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd '/content/drive/MyDrive/Thesis_UU/push/2group'

Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import re

Load csv file

In [ ]:
papers = pd.read_csv('Data (CSV)/preprocessing_part_1_common.csv', encoding="utf-8-sig")

In [ ]:
papers.Journal_Name.tail(5)

#### Preprocessing

Retracted word removal

In [ ]:
papers.Text = papers.Text.str.replace('RETRACTION|R E T R A C T I O N|Retraction|retraction|Retracted|retracted|RETRACTED|R E T R A C T E D|Retract|retract|WITHDRAWN|W I T H D R A W N|Withdrawn|withdrawn|WITHDRAW|W I T H D R A W|Withdraw|withdraw', '', regex = True)

Number removal

In [ ]:
papers.Text = papers.Text.str.replace(r'\d', '', regex = True)

Proper noun, Space, Stop word, Punctuation removal + lowercasing & lemmatization

In [ ]:
import spacy 
nlp = spacy.load("en_core_web_sm") #english
texts = list(papers.Text.values)
processed_texts = [text for text in nlp.pipe(texts, disable=["ner", "parser"])] #setting up pipe for preprocessing

In [ ]:
preprocessed_texts = [[word.lemma_.lower() for word in processed_text if not word.is_stop and not word.is_punct and not word.pos_ == "PROPN" and not word.is_space] 
                      for processed_text in processed_texts]

In [ ]:
papers['preprocessed'] = preprocessed_texts #adding columns with preprocessed words to df

In [ ]:
papers['preprocessed'].head(2)

In [ ]:
import re
papers = papers.astype({"preprocessed": str}, errors='raise') 
papers['preprocessed'] = papers['preprocessed'].replace({',': ''}, regex = True)
papers['preprocessed'] = papers['preprocessed'].replace({'\'': ''}, regex = True)
papers['preprocessed'] = papers['preprocessed'].replace({'\[': ''}, regex = True)
papers['preprocessed'] = papers['preprocessed'].replace({'\]': ''}, regex = True)

In [ ]:
papers['preprocessed'].head(2)

In [ ]:
papers.head(2)

In [ ]:
papers.columns

#### Check group sizes

In [ ]:
len(papers)

In [ ]:
len(papers[papers['Retracted'] == 0])

In [ ]:
len(papers[papers['Retracted'] == 1])

In [ ]:
grouped = papers.groupby(['Journal_Name', 'Retracted', ]).count().reset_index()

In [ ]:
grouped.sort_values('Unnamed: 0', ascending=False) 
#checking how much articles belong to each journal for each class

In [ ]:
papers['Journal_Name'].value_counts(ascending = False) #checking how much articles belong to each journal

#### Creating datasets for training/testing and external validation

Note: so we can also see if classifiers are accurate for distinguishing on retraction for journals they are not trained on

In [ ]:
test_journals = ['plos one', 'journal of fundamental and applied sciences'] #journals for external validation

In [ ]:
journal_test_data_set = papers[papers.Journal_Name.isin(test_journals)] #external validation dataset
journal_train_data_set = papers[~papers.Journal_Name.isin(test_journals)] #train/test dataset

In [ ]:
len(journal_test_data_set)

In [ ]:
len(journal_train_data_set)

In [ ]:
journal_test_data_set.groupby(['Journal_Name', 'Retracted', ]).size()
#check groups per journal for external validation dataset

In [ ]:
journal_train_data_set.groupby(['Journal_Name', 'Retracted', ]).size()
#check groups per journal for train/test dataset

In [ ]:
len(journal_test_data_set)

In [ ]:
classes = journal_test_data_set.groupby(['Retracted']).count().reset_index()
classes
#group size for external validation dataset

In [ ]:
len(set(journal_test_data_set.Journal_Name)) #amount of journal for external validation dataset

In [ ]:
set(journal_test_data_set.Journal_Name) #journal names for external validation dataset

In [ ]:
len(journal_train_data_set)

In [ ]:
classes = journal_train_data_set.groupby(['Retracted']).count().reset_index()
classes
#group size for train/test dataset

In [ ]:
len(set(journal_train_data_set.Journal_Name)) #amount of journal for train/test dataset

In [ ]:
set(journal_train_data_set.Journal_Name) #journal names for train/test dataset

In [ ]:
papers = pd.concat([journal_train_data_set, journal_test_data_set])

In [ ]:
papers.head(2)

In [ ]:
len(papers)

In [ ]:
classes = papers.groupby(['Retracted']).count().reset_index()
classes

Save datasets

In [ ]:
journal_test_data_set.to_csv('Data (CSV)/journal_test_data_set_common.csv')
journal_train_data_set.to_csv('Data (CSV)/journal_train_data_set_common.csv')